In [1]:
%matplotlib notebook

이 장의 예제는 아래의 링크에서 아이디어를 얻어서 새로 구현했습니다.

### reference: https://towardsdatascience.com/attention-seq2seq-with-pytorch-learning-to-invert-a-sequence-34faf4133e53

In [2]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from tqdm import tqdm
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence

In [3]:
torch.__version__

'1.7.0+cu101'

In [4]:
torch.__version__

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
MAX_LENGTH = 15

In [6]:
bos = 0
eos = 1
pad = 2

In [7]:
inverse_map = {
    'a':'z',
    'b':'y',
    'c':'x',
    'd':'w',
    'e':'v',
    'f':'u',
    'g':'t',
    'h':'s',
    'i':'r',
    'j':'q',
    'k':'p',
    'l':'o',
    'm':'n',
    'n':'m',
    'o':'l',
    'p':'k',
    'q':'j',
    'r':'i',
    's':'h',
    't':'g',
    'u':'f',
    'v':'e',
    'w':'d',
    'x':'c',
    'y':'b',
    'z':'a'
}

In [8]:
a2i = {
    '<s>':0,
    '</s>':1,
    '<pad>':2,
    'a':3,
    'b':4,
    'c':5,
    'd':6,
    'e':7,
    'f':8,
    'g':9,
    'h':10,
    'i':11,
    'j':12,
    'k':13,
    'l':14,
    'm':15,
    'n':16,
    'o':17,
    'p':18,
    'q':19,
    'r':20,
    's':21,
    't':22,
    'u':23,
    'v':24,
    'w':25,
    'x':26,
    'y':27,
    'z':28,
}

In [9]:
i2a = {v:k for k, v in a2i.items()}

In [10]:
i2a

{0: '<s>',
 1: '</s>',
 2: '<pad>',
 3: 'a',
 4: 'b',
 5: 'c',
 6: 'd',
 7: 'e',
 8: 'f',
 9: 'g',
 10: 'h',
 11: 'i',
 12: 'j',
 13: 'k',
 14: 'l',
 15: 'm',
 16: 'n',
 17: 'o',
 18: 'p',
 19: 'q',
 20: 'r',
 21: 's',
 22: 't',
 23: 'u',
 24: 'v',
 25: 'w',
 26: 'x',
 27: 'y',
 28: 'z'}

In [11]:
def generate_random_alphabet_index():
    random_length = np.random.randint(10, MAX_LENGTH-2)    # -2 because of <s> and </s>
    random_alphabet_index = np.random.randint(0, 26, random_length) + 3
    return random_alphabet_index.tolist(), random_length

In [12]:
def make_alphabet_tensor():
    indice, length = generate_random_alphabet_index()
    indice = indice + [pad]*(MAX_LENGTH-length)
    return torch.tensor([indice]).to(device)

In [13]:
class AlphabetToyDataset(Dataset):
    def __init__(self, n_dataset=1000):
        bos = 0
        eos = 1
        pad = 2
        self.inputs = []
        self.labels = []
        self.length = []
        for _ in range(n_dataset):
            # make input example
            aindex, alen = generate_random_alphabet_index()
            
            # index to alphabet
            #alphabet = list(map(lambda a: i2a[a], aindex))
            
            # inversing
            #inversed_alphabet = list(map(lambda a: inverse_map[a], alphabet))
            
            # alphabet to index
            #iindex = list(map(lambda ia: a2i[ia], inversed_alphabet))
            iindex = aindex[::-1]
            
            # add bos, eos and pad
            n_pad = MAX_LENGTH - len(aindex) - 1
            aindex = aindex + [eos] + [pad]*n_pad
            iindex = iindex + [eos] + [pad]*n_pad
            
            # add to examples
            self.inputs.append(aindex)
            self.labels.append(iindex)
            self.length.append(alen)
            
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, index):
        return [
            torch.tensor(self.inputs[index], dtype=torch.long),
            torch.tensor(self.labels[index], dtype=torch.long),
            torch.tensor(self.length[index], dtype=torch.long)
        ]

In [15]:
train_dataset = AlphabetToyDataset(n_dataset=3000)
valid_dataset = AlphabetToyDataset(n_dataset=300)

In [16]:
len(train_dataset), len(valid_dataset)

(3000, 300)

In [17]:
def convert_index_to_alphabet(index):
    alphabet = list(map(lambda i: i2a[i], index))
    return ' '.join(alphabet)

In [18]:
for i in range(3):
    ex = train_dataset[i]
    aindex, iindex, length = ex
    
    print('aindex_{}: {}'.format(len(aindex), convert_index_to_alphabet(aindex.numpy())))
    print('iindex_{}: {}'.format(len(iindex), convert_index_to_alphabet(iindex.numpy())))
    print('** aindex_{}: {}'.format(len(aindex), aindex))
    print('** iindex_{}: {}'.format(len(iindex), iindex))
    print('------------')

aindex_15: w k u p t k v z l u </s> <pad> <pad> <pad> <pad>
iindex_15: u l z v k t p u k w </s> <pad> <pad> <pad> <pad>
** aindex_15: tensor([25, 13, 23, 18, 22, 13, 24, 28, 14, 23,  1,  2,  2,  2,  2])
** iindex_15: tensor([23, 14, 28, 24, 13, 22, 18, 23, 13, 25,  1,  2,  2,  2,  2])
------------
aindex_15: s t z y n p b b a e </s> <pad> <pad> <pad> <pad>
iindex_15: e a b b p n y z t s </s> <pad> <pad> <pad> <pad>
** aindex_15: tensor([21, 22, 28, 27, 16, 18,  4,  4,  3,  7,  1,  2,  2,  2,  2])
** iindex_15: tensor([ 7,  3,  4,  4, 18, 16, 27, 28, 22, 21,  1,  2,  2,  2,  2])
------------
aindex_15: y k d q l d q e u y </s> <pad> <pad> <pad> <pad>
iindex_15: y u e q d l q d k y </s> <pad> <pad> <pad> <pad>
** aindex_15: tensor([27, 13,  6, 19, 14,  6, 19,  7, 23, 27,  1,  2,  2,  2,  2])
** iindex_15: tensor([27, 23,  7, 19,  6, 14, 19,  6, 13, 27,  1,  2,  2,  2,  2])
------------


In [19]:
def collate_fn(batch):
    inputs = pad_sequence([b[0] for b in batch], batch_first=True)
    targets = pad_sequence([b[1] for b in batch], batch_first=True)
    lengths = torch.stack([b[2] for b in batch])

    lengths, indice = torch.sort(lengths, descending=True)
    inputs = inputs[indice]
    targets = targets[indice]
    return inputs, targets, lengths

In [20]:
train_dataloader = DataLoader(train_dataset, collate_fn=collate_fn, batch_size=16)
valid_dataloader = DataLoader(valid_dataset, collate_fn=collate_fn, batch_size=1)

In [21]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)

    def forward(self, inputs, hidden):
        '''
        Input Parameters
        - inputs: (B,M)
        - hidden: (1,B,H)
        
        Output returns
        - output: (B,M,H)
        - hidden: (1,B,H)
        
        Logging outputs
        ** inputs: torch.Size([16, 5])
        ** hidden: torch.Size([1, 16, 256])
        ** embedded: torch.Size([16, 5, 256])
        ** final-output: torch.Size([16, 5, 256])
        ** final-hidden: torch.Size([1, 16, 256])
        '''
        #print('** inputs: {}'.format(inputs.shape))
        #print('** hidden: {}'.format(hidden.shape))
        embedded = self.embedding(inputs)    # (B,M,H)
        #print('** embedded: {}'.format(embedded.shape))
        output, hidden = self.gru(embedded, hidden)    # (B,M,H), (1,B,H)
        #print('** final-output: {}'.format(output.shape))
        #print('** final-hidden: {}'.format(hidden.shape))
        return output, hidden

    def initHidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)

In [22]:
hidden_size = 256
enc = EncoderRNN(26+3, hidden_size).to(device)

In [23]:
inp = torch.from_numpy(np.array([
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
]))
inp = inp.to(device)
hid = enc.initHidden(inp.size(0))

In [24]:
o, h = enc(inp, hid)
o.shape, h.shape

(torch.Size([16, 5, 256]), torch.Size([1, 16, 256]))

In [25]:
# without Attention
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        #print('DecoderRNN_INPUT: input: {} → {:.4f}'.format(input.shape, input.sum()))
        #print('DecoderRNN_INPUT: hidden: {} → {:.4f}'.format(hidden.shape, hidden.sum()))
        
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        #print('** output: {} → {:.4f}'.format(output.shape, output.sum()))
        
        
        output, hidden = self.gru(output, hidden)
        #print('** gru-output: {} → {:.4f}'.format(output.shape, output.sum()))
        #print('** gru-hidden: {} → {:.4f}'.format(hidden.shape, hidden.sum()))
        
        output = self.softmax(self.out(output[0]))
        #print('** hidden: {}'.format(hidden.shape))
        
        #print('DecoderRNN_OUTPUT: output: {} → {:.4f}'.format(output.shape, output.sum()))
        #print('DecoderRNN_OUTPUT: hidden: {} → {:.4f}'.format(hidden.shape, hidden.sum()))
        return output, hidden

    def initHidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)

In [26]:
# with attention
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size, batch_first=True)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, inputs, hidden, encoder_outputs):
        '''
        Input Parameters
        - inputs: (B,1)
        - hidden: (1,B,H)
        - encoder_outputs: (B,M,H)
        
        Output returns
        - output: (B,1,O)
        - hidden: (B,1,H)
        - attn_weights: (B,1,M)
        
        Logging outputs
        ** inputs: torch.Size([16, 1])
        ** hidden: torch.Size(v[1, 16, 256])
        ** encoder_outputs: torch.Size([16, 15, 256])
        ** embedded: torch.Size([16, 1, 256])
        ** attn_weights: torch.Size([16, 1, 15])
        ** attn_weights: tensor([[[0.1068, 0.0260, 0.0466, 0.0421, 0.0885, 0.0843, 0.0718, 0.0575,
                  0.0361, 0.0273, 0.0339, 0.1882, 0.0606, 0.0723, 0.0581]],

                [[0.1634, 0.0374, 0.0518, 0.0382, 0.0771, 0.0577, 0.0624, 0.0658,
                  0.0286, 0.0351, 0.0336, 0.1435, 0.0603, 0.0940, 0.0510]],

                [[0.1191, 0.0284, 0.0447, 0.0361, 0.0816, 0.0879, 0.0675, 0.0523,
                  0.0293, 0.0273, 0.0535, 0.1981, 0.0588, 0.0728, 0.0427]],

                [[0.1043, 0.0357, 0.0479, 0.0426, 0.1006, 0.0658, 0.0556, 0.0538,
                  0.0372, 0.0476, 0.0348, 0.1877, 0.0641, 0.0705, 0.0516]],

                [[0.1179, 0.0354, 0.0404, 0.0565, 0.0785, 0.0645, 0.0700, 0.0685,
                  0.0380, 0.0414, 0.0456, 0.1519, 0.0710, 0.0682, 0.0523]],

                [[0.1363, 0.0500, 0.0404, 0.0383, 0.0748, 0.0674, 0.0733, 0.0520,
                  0.0288, 0.0306, 0.0466, 0.1517, 0.0769, 0.0932, 0.0395]],

                [[0.1429, 0.0315, 0.0380, 0.0411, 0.0754, 0.0555, 0.0701, 0.0626,
                  0.0337, 0.0311, 0.0398, 0.1486, 0.0813, 0.0885, 0.0599]],

                [[0.1183, 0.0333, 0.0422, 0.0445, 0.0842, 0.0600, 0.0854, 0.0732,
                  0.0299, 0.0447, 0.0299, 0.1091, 0.0865, 0.0831, 0.0757]],

                [[0.1145, 0.0353, 0.0372, 0.0386, 0.0867, 0.0568, 0.0840, 0.0514,
                  0.0393, 0.0285, 0.0397, 0.1800, 0.0759, 0.0755, 0.0565]],

                [[0.1614, 0.0251, 0.0357, 0.0470, 0.0903, 0.0756, 0.0521, 0.0419,
                  0.0305, 0.0227, 0.0343, 0.1683, 0.0643, 0.0891, 0.0616]],

                [[0.1152, 0.0248, 0.0413, 0.0535, 0.0836, 0.0599, 0.0707, 0.0559,
                  0.0333, 0.0394, 0.0348, 0.1688, 0.0728, 0.0880, 0.0582]],

                [[0.1475, 0.0310, 0.0462, 0.0446, 0.0865, 0.0484, 0.0775, 0.0775,
                  0.0378, 0.0357, 0.0360, 0.1070, 0.0689, 0.1070, 0.0485]],

                [[0.1168, 0.0299, 0.0347, 0.0407, 0.0787, 0.0605, 0.0683, 0.0517,
                  0.0312, 0.0334, 0.0426, 0.1895, 0.0739, 0.0914, 0.0565]],

                [[0.1122, 0.0315, 0.0371, 0.0409, 0.1061, 0.0805, 0.0861, 0.0475,
                  0.0399, 0.0262, 0.0330, 0.1467, 0.0676, 0.0788, 0.0660]],

                [[0.1491, 0.0363, 0.0410, 0.0398, 0.0919, 0.0586, 0.0605, 0.0647,
                  0.0378, 0.0277, 0.0356, 0.1396, 0.0699, 0.0981, 0.0495]],

                [[0.1127, 0.0238, 0.0486, 0.0385, 0.0809, 0.0726, 0.0829, 0.0495,
                  0.0441, 0.0275, 0.0412, 0.1654, 0.0627, 0.0825, 0.0672]]],
               device='cuda:0', grad_fn=<SoftmaxBackward>)
        ** attn_weights.sum(): torch.Size([])
        ** attn_weights.sum(): 16.0000
        ** attn_applied: torch.Size([16, 1, 256])
        ** output: torch.Size([16, 1, 512])
        ** output: torch.Size([16, 1, 256])
        ** output: torch.Size([16, 1, 256])
        ** hidden: torch.Size([1, 16, 256])
        ** gru-output: torch.Size([16, 1, 256])
        ** gru-hidden: torch.Size([1, 16, 256])
        ** final-output: torch.Size([16, 1, 29])
        ** final-hidden: torch.Size([1, 16, 256])
        ** final-attn_weights: torch.Size([16, 1, 15])
        '''
        embedded = self.embedding(inputs)    # (B,1,H)
        embedded = self.dropout(embedded)
        
        # query: embedded
        # key: hidden
        # value: encoder_outputs
        
        attn_weights = F.softmax(
            self.attn(
                torch.cat((embedded, hidden.transpose(0, 1)), -1)    # (B,1,2H)
            ),    # (B,1,M)
            dim=-1)    # (B,1,M)
        #print('** attn_weights: {}'.format(attn_weights.shape))
        #print('** attn_weights: {}'.format(attn_weights))
        #print('** attn_weights.sum(): {}'.format(attn_weights.sum().shape))
        #print('** attn_weights.sum(): {:.4f}'.format(attn_weights.sum()))
        
        attn_applied = torch.bmm(attn_weights, encoder_outputs)    # (B,1,H)
        #print('** attn_applied: {}'.format(attn_applied.shape))
        #return

        output = torch.cat((embedded, attn_applied), -1)    # (B,1,2H)
        #print('** output: {}'.format(output.shape))
        output = self.attn_combine(output)    # (B,1,H)
        #print('** output: {}'.format(output.shape))    # (B,1,H)
        
        output = F.relu(output)
        #print('** output: {}'.format(output.shape))    # (B,1,H)
        #print('** hidden: {}'.format(hidden.shape))    # (B,1,H)
        output, hidden = self.gru(output, hidden)    # (B,1,H) (1,B,H)
        #print('** gru-output: {}'.format(output.shape))
        #print('** gru-hidden: {}'.format(hidden.shape))

        output = F.log_softmax(self.out(output), dim=-1)
        #print('** final-output: {}'.format(output.shape))
        #print('** final-hidden: {}'.format(hidden.shape))
        #print('** final-attn_weights: {}'.format(attn_weights.shape))
        return output, hidden, attn_weights

    def initHidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)

In [27]:
teacher_forcing_ratio = 0.5

In [28]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH, with_attention=True):
    batch_size = input_tensor.size(0)
    encoder_hidden = encoder.initHidden(batch_size)

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(1)
    target_length = target_tensor.size(1)
    
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    loss = 0

    encoder_outputs, encoder_hidden = encoder(input_tensor, encoder_hidden)
    
    decoder_input = torch.tensor([bos]*batch_size, device=device)
    decoder_input = decoder_input.unsqueeze(-1)    # (B,1)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if np.random.random() < teacher_forcing_ratio else False
    
    for di in range(target_length):
        decoder_output, decoder_hidden, decoder_attention = decoder(
            decoder_input,    # (B,1)
            decoder_hidden,   # (1,B,H)
            encoder_outputs   # (B,M,H)
        )
        decoder_output = decoder_output.squeeze(1)
        
        loss += criterion(decoder_output, target_tensor[:,di])
        
        if use_teacher_forcing:
            decoder_input = target_tensor[:,di].unsqueeze(-1)    # (B,1)
        else:
            topv, topi = decoder_output.topk(1)
            decoder_input = topi    # (B,1)

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [29]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [30]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01, with_attention=True):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # print_every 마다 초기화
    plot_loss_total = 0  # plot_every 마다 초기화

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for iter, batch in enumerate(train_dataloader):
        input_tensor, target_tensor, length_tensor = batch
        input_tensor = input_tensor.to(device)
        target_tensor = target_tensor.to(device)
        length_tensor = length_tensor.to(device)
        
        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion, with_attention=with_attention)
        print_loss_total += loss
        plot_loss_total += loss

        if (iter+1) % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, (iter+1) / n_iters),
                                         (iter+1), (iter+1) / n_iters * 100, print_loss_avg))

        if (iter+1) % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [31]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # 주기적인 간격에 이 locator가 tick을 설정
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [32]:
def evaluate(encoder, decoder, input_tensor, max_length=MAX_LENGTH):
    with torch.no_grad():
        batch_size = input_tensor.size(0)
        encoder_hidden = encoder.initHidden(batch_size)
        #print(input_tensor.shape, encoder_hidden.shape)
        encoder_outputs, encoder_hidden = encoder(input_tensor, encoder_hidden)
        
        decoder_input = torch.tensor([bos]*batch_size, device=device)
        decoder_input = decoder_input.unsqueeze(-1)    # (B,1)
        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            #print('** ', decoder_input.shape, decoder_hidden.shape, encoder_outputs.shape)
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input,    # (B,1)
                decoder_hidden,   # (1,B,H)
                encoder_outputs   # (B,M,H)
            )
            decoder_output = decoder_output.squeeze(1)
            decoder_attentions[:, di] = decoder_attention
            
            topv, topi = decoder_output.topk(1)
            decoder_input = topi    # (B,1)
            if topi.item() == eos:
                decoded_words.append('</s>')
                break
            else:
                decoded_words.append(i2a[topi.item()])

        return decoded_words, decoder_attentions[:di + 1]

In [33]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [34]:
for t in train_dataloader:
    print(t)
    break

(tensor([[ 3, 12, 19, 28, 20, 10, 19,  5, 14, 23, 23, 14,  1,  2,  2],
        [16, 25, 21, 14, 27, 21, 12, 19, 20, 11, 24, 12,  1,  2,  2],
        [13,  5, 18, 24, 16, 16, 26, 22, 13, 17, 23, 10,  1,  2,  2],
        [23,  7,  5, 19, 25, 20,  7, 16, 16, 18, 11,  9,  1,  2,  2],
        [ 6, 27,  8, 13, 25, 18,  3, 26, 21, 22, 24,  8,  1,  2,  2],
        [ 4, 15, 26, 23, 12, 15,  6, 21,  7, 11, 21, 23,  1,  2,  2],
        [28,  7, 18, 20, 28, 24,  9, 19, 15, 13, 12, 26,  1,  2,  2],
        [ 7, 18, 15, 20,  8, 11, 14, 22,  5, 21, 24,  1,  2,  2,  2],
        [ 4, 22, 10, 21, 24, 27, 28, 14, 21,  8, 27,  1,  2,  2,  2],
        [ 3,  6, 22, 15,  8, 10, 21, 10,  4,  7,  7,  1,  2,  2,  2],
        [16,  5,  4, 21, 16,  8, 21, 18, 15, 11, 26,  1,  2,  2,  2],
        [25, 13, 23, 18, 22, 13, 24, 28, 14, 23,  1,  2,  2,  2,  2],
        [21, 22, 28, 27, 16, 18,  4,  4,  3,  7,  1,  2,  2,  2,  2],
        [27, 13,  6, 19, 14,  6, 19,  7, 23, 27,  1,  2,  2,  2,  2],
        [10, 27, 21

In [35]:
hidden_size = 256
encoder1 = EncoderRNN(26+3, hidden_size).to(device)
decoder1 = AttnDecoderRNN(hidden_size, 26+3, dropout_p=0.1).to(device)

In [36]:
# hidden_size = 256
# encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
# decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)

In [37]:
#### Input validation

In [38]:
# trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

In [39]:
for _ in range(15):
    trainIters(encoder1, decoder1, 75000, print_every=30, with_attention=True)

0m 1s (- 79m 58s) (30 0%) 3.0960
0m 3s (- 72m 25s) (60 0%) 2.8949
0m 5s (- 69m 35s) (90 0%) 2.8327
0m 6s (- 67m 56s) (120 0%) 2.7822
0m 8s (- 67m 3s) (150 0%) 2.7563
0m 9s (- 66m 24s) (180 0%) 2.6831
0m 1s (- 62m 9s) (30 0%) 2.6578
0m 3s (- 62m 28s) (60 0%) 2.6455
0m 4s (- 62m 10s) (90 0%) 2.6601
0m 5s (- 62m 20s) (120 0%) 2.6389
0m 7s (- 62m 22s) (150 0%) 2.6329
0m 9s (- 62m 28s) (180 0%) 2.5795
0m 1s (- 61m 42s) (30 0%) 2.5696
0m 2s (- 61m 59s) (60 0%) 2.5504
0m 4s (- 62m 35s) (90 0%) 2.5644
0m 6s (- 62m 35s) (120 0%) 2.5287
0m 7s (- 62m 37s) (150 0%) 2.5176
0m 9s (- 62m 44s) (180 0%) 2.5044
0m 1s (- 63m 19s) (30 0%) 2.4489
0m 3s (- 63m 13s) (60 0%) 2.4004
0m 4s (- 62m 48s) (90 0%) 2.3041
0m 6s (- 62m 57s) (120 0%) 2.2643
0m 7s (- 62m 47s) (150 0%) 2.0685
0m 9s (- 62m 46s) (180 0%) 2.0131
0m 1s (- 62m 13s) (30 0%) 1.7731
0m 2s (- 62m 12s) (60 0%) 1.5341
0m 4s (- 62m 33s) (90 0%) 1.5236
0m 6s (- 62m 28s) (120 0%) 1.2481
0m 7s (- 62m 22s) (150 0%) 1.2174
0m 8s (- 62m 17s) (180 0%) 1.20

/heavy_data/jkfirst/workspace/git/publish/env_pub/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


0m 1s (- 63m 16s) (30 0%) 0.1914
0m 2s (- 62m 25s) (60 0%) 0.1904
0m 4s (- 61m 33s) (90 0%) 0.1858
0m 5s (- 61m 23s) (120 0%) 0.2020
0m 7s (- 61m 31s) (150 0%) 0.1985
0m 8s (- 61m 47s) (180 0%) 0.1800
0m 1s (- 62m 46s) (30 0%) 0.1757
0m 3s (- 62m 50s) (60 0%) 0.1838
0m 4s (- 62m 31s) (90 0%) 0.1623
0m 6s (- 62m 32s) (120 0%) 0.1732
0m 7s (- 62m 23s) (150 0%) 0.1658
0m 9s (- 62m 25s) (180 0%) 0.1763
0m 1s (- 61m 54s) (30 0%) 0.1529
0m 2s (- 61m 44s) (60 0%) 0.1454
0m 4s (- 61m 59s) (90 0%) 0.1612
0m 5s (- 61m 57s) (120 0%) 0.1399
0m 7s (- 62m 9s) (150 0%) 0.1524
0m 8s (- 62m 13s) (180 0%) 0.1434
0m 1s (- 62m 40s) (30 0%) 0.1232
0m 3s (- 63m 1s) (60 0%) 0.1377
0m 4s (- 62m 48s) (90 0%) 0.1390
0m 6s (- 62m 51s) (120 0%) 0.1361
0m 7s (- 62m 47s) (150 0%) 0.1338
0m 9s (- 62m 39s) (180 0%) 0.1285


In [40]:
for _ in range(5):
    trainIters(encoder1, decoder1, 75000, print_every=30, with_attention=True)

0m 1s (- 61m 5s) (30 0%) 0.1160
0m 2s (- 61m 41s) (60 0%) 0.1244
0m 4s (- 62m 9s) (90 0%) 0.1191
0m 5s (- 62m 15s) (120 0%) 0.1200
0m 7s (- 62m 16s) (150 0%) 0.1260
0m 9s (- 62m 32s) (180 0%) 0.1153
0m 1s (- 62m 58s) (30 0%) 0.4366
0m 3s (- 62m 55s) (60 0%) 0.8851
0m 4s (- 62m 57s) (90 0%) 0.1609
0m 6s (- 62m 41s) (120 0%) 0.1312
0m 7s (- 62m 29s) (150 0%) 0.1323
0m 9s (- 62m 44s) (180 0%) 0.1166
0m 1s (- 63m 14s) (30 0%) 0.1073
0m 3s (- 63m 2s) (60 0%) 0.1088
0m 4s (- 63m 6s) (90 0%) 0.1032
0m 6s (- 62m 56s) (120 0%) 0.1090
0m 7s (- 63m 1s) (150 0%) 0.1091
0m 9s (- 63m 6s) (180 0%) 0.1085
0m 1s (- 63m 40s) (30 0%) 0.0917
0m 3s (- 63m 42s) (60 0%) 0.0921
0m 4s (- 63m 24s) (90 0%) 0.0894
0m 6s (- 62m 57s) (120 0%) 0.0852
0m 7s (- 63m 0s) (150 0%) 0.0946
0m 9s (- 63m 12s) (180 0%) 0.0870
0m 1s (- 63m 43s) (30 0%) 0.0815
0m 3s (- 63m 13s) (60 0%) 0.0798
0m 4s (- 63m 35s) (90 0%) 0.0784
0m 6s (- 63m 27s) (120 0%) 0.0809
0m 7s (- 63m 15s) (150 0%) 0.0817
0m 9s (- 63m 18s) (180 0%) 0.0882


In [41]:
# trainIters(encoder1, decoder1, 75000, print_every=100, with_attention=True)

In [42]:
for d in valid_dataloader:
    print(d[0].shape)
    break

torch.Size([1, 15])


In [43]:
for d in valid_dataloader:
    input_tensor, target_tensor, length_tensor = d
    input_string = list(map(lambda i: i2a[i], input_tensor.numpy()[0]))
    output_string = list(map(lambda i: i2a[i], target_tensor.numpy()[0]))
    input_string = np.array(input_string)
    output_string = np.array(output_string)
    print('input: {}'.format(input_string))
    print('output: {}'.format(output_string))
    
    input_tensor = input_tensor.to(device)
    target_tensor = target_tensor.to(device)
    length_tensor = length_tensor.to(device)
    
    pred, attn_weight = evaluate(encoder1, decoder1, input_tensor)
    pred = np.array(pred)
    print('pred: {}'.format(pred))
    print('-------------')
    #break

input: ['b' 'u' 'l' 'g' 'h' 'j' 'i' 'b' 'm' 'r' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
output: ['r' 'm' 'b' 'i' 'j' 'h' 'g' 'l' 'u' 'b' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
pred: ['r' 'm' 'b' 'i' 'j' 'h' 'g' 'l' 'u' 'b' '</s>']
-------------
input: ['w' 'f' 'd' 'l' 'n' 's' 'c' 'w' 'h' 'g' 'v' '</s>' '<pad>' '<pad>'
 '<pad>']
output: ['v' 'g' 'h' 'w' 'c' 's' 'n' 'l' 'd' 'f' 'w' '</s>' '<pad>' '<pad>'
 '<pad>']
pred: ['v' 'g' 'h' 'w' 'c' 's' 'n' 'l' 'd' 'f' 'w' '</s>']
-------------
input: ['j' 'e' 'q' 'p' 'b' 'd' 'r' 'x' 'v' 'p' 'e' '</s>' '<pad>' '<pad>'
 '<pad>']
output: ['e' 'p' 'v' 'x' 'r' 'd' 'b' 'p' 'q' 'e' 'j' '</s>' '<pad>' '<pad>'
 '<pad>']
pred: ['e' 'p' 'x' 'x' 'r' 'd' 'b' 'p' 'q' 'e' 'j' '</s>']
-------------
input: ['m' 'e' 'd' 'k' 'm' 'o' 'z' 'n' 'b' 'b' 'm' 'q' '</s>' '<pad>' '<pad>']
output: ['q' 'm' 'b' 'b' 'n' 'z' 'o' 'm' 'k' 'd' 'e' 'm' '</s>' '<pad>' '<pad>']
pred: ['q' 'b' 'm' 'm' 'n' 'z' 'o' 'm' 'k' 'd' 'e' 'm' '</s>']
-------------
input: ['s' 'k' 'b' 'f' 'q

pred: ['y' 'k' 'f' 'r' 'l' 'c' 'g' 'q' 'e' 'k' 'm' '</s>']
-------------
input: ['h' 'l' 'k' 'w' 'n' 'c' 'g' 'r' 'c' 'i' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
output: ['i' 'c' 'r' 'g' 'c' 'n' 'w' 'k' 'l' 'h' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
pred: ['i' 'c' 'r' 'g' 'c' 'n' 'w' 'k' 'l' 'h' '</s>']
-------------
input: ['b' 'a' 'h' 'x' 'n' 'a' 'c' 'r' 'n' 'o' 'd' '</s>' '<pad>' '<pad>'
 '<pad>']
output: ['d' 'o' 'n' 'r' 'c' 'a' 'n' 'x' 'h' 'a' 'b' '</s>' '<pad>' '<pad>'
 '<pad>']
pred: ['d' 'o' 'n' 'r' 'c' 'a' 'n' 'x' 'h' 'a' 'b' '</s>']
-------------
input: ['b' 'x' 'o' 'o' 'o' 'k' 'i' 'r' 'l' 'a' 'o' '</s>' '<pad>' '<pad>'
 '<pad>']
output: ['o' 'a' 'l' 'r' 'i' 'k' 'o' 'o' 'o' 'x' 'b' '</s>' '<pad>' '<pad>'
 '<pad>']
pred: ['o' 'a' 'l' 'r' 'i' 'k' 'o' 'o' 'o' 'x' 'b' '</s>']
-------------
input: ['w' 'r' 'e' 'm' 'c' 'd' 'b' 'k' 'h' 'u' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
output: ['u' 'h' 'k' 'b' 'd' 'c' 'm' 'e' 'r' 'w' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
pred: ['u' '

pred: ['h' 'd' 'c' 'g' 'c' 't' 'b' 't' 'u' 's' 'w' '</s>']
-------------
input: ['b' 'v' 'z' 'r' 'g' 'l' 'a' 'u' 'o' 'z' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
output: ['z' 'o' 'u' 'a' 'l' 'g' 'r' 'z' 'v' 'b' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
pred: ['z' 'o' 'u' 'a' 'l' 'g' 'r' 'z' 'v' 'b' '</s>']
-------------
input: ['p' 'y' 'a' 'c' 'v' 'd' 'g' 'j' 'o' 'i' 'j' '</s>' '<pad>' '<pad>'
 '<pad>']
output: ['j' 'i' 'o' 'j' 'g' 'd' 'v' 'c' 'a' 'y' 'p' '</s>' '<pad>' '<pad>'
 '<pad>']
pred: ['j' 'i' 'o' 'j' 'g' 'd' 'v' 'c' 'a' 'y' 'p' '</s>']
-------------
input: ['t' 'a' 'w' 'l' 'p' 'c' 'x' 's' 't' 'j' 'c' '</s>' '<pad>' '<pad>'
 '<pad>']
output: ['c' 'j' 't' 's' 'x' 'c' 'p' 'l' 'w' 'a' 't' '</s>' '<pad>' '<pad>'
 '<pad>']
pred: ['c' 'j' 't' 's' 'x' 'c' 'p' 'l' 'w' 'a' 't' '</s>']
-------------
input: ['u' 'b' 'j' 'a' 'y' 'd' 'a' 'a' 'p' 'e' 'o' 'y' '</s>' '<pad>' '<pad>']
output: ['y' 'o' 'e' 'p' 'a' 'a' 'd' 'y' 'a' 'j' 'b' 'u' '</s>' '<pad>' '<pad>']
pred: ['y' 'o' 'e' 'o' 'a' 'a'

pred: ['v' 'g' 'v' 'n' 'g' 'z' 'r' 'l' 'v' 'e' 'q' '</s>']
-------------
input: ['r' 'r' 'c' 'n' 'd' 't' 'k' 'a' 'u' 'q' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
output: ['q' 'u' 'a' 'k' 't' 'd' 'n' 'c' 'r' 'r' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
pred: ['q' 'u' 'a' 'k' 't' 'd' 'n' 'c' 'r' 'r' '</s>']
-------------
input: ['f' 'b' 'b' 'c' 'w' 'l' 'z' 'k' 'f' 't' 'l' '</s>' '<pad>' '<pad>'
 '<pad>']
output: ['l' 't' 'f' 'k' 'z' 'l' 'w' 'c' 'b' 'b' 'f' '</s>' '<pad>' '<pad>'
 '<pad>']
pred: ['l' 't' 'f' 'k' 'z' 'l' 'w' 'c' 'b' 'b' 'f' '</s>']
-------------
input: ['p' 'f' 'm' 'r' 'j' 'v' 'g' 'y' 's' 'r' 's' '</s>' '<pad>' '<pad>'
 '<pad>']
output: ['s' 'r' 's' 'y' 'g' 'v' 'j' 'r' 'm' 'f' 'p' '</s>' '<pad>' '<pad>'
 '<pad>']
pred: ['s' 'r' 's' 'y' 'g' 'v' 'j' 'r' 'm' 'f' 'p' '</s>']
-------------
input: ['f' 'x' 'q' 'a' 'r' 'm' 'y' 'b' 'r' 'y' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
output: ['y' 'r' 'b' 'y' 'm' 'r' 'a' 'q' 'x' 'f' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
pred: ['y' '

pred: ['t' 'y' 'l' 'o' 't' 'p' 's' 'y' 'm' 'w' '</s>']
-------------
input: ['g' 'y' 'l' 'k' 't' 'o' 's' 'h' 'n' 'u' 'b' '</s>' '<pad>' '<pad>'
 '<pad>']
output: ['b' 'u' 'n' 'h' 's' 'o' 't' 'k' 'l' 'y' 'g' '</s>' '<pad>' '<pad>'
 '<pad>']
pred: ['b' 'u' 'h' 'h' 's' 'o' 't' 'k' 'l' 'y' 'g' '</s>']
-------------
input: ['i' 'a' 'w' 'm' 'q' 'q' 'g' 'e' 'j' 'x' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
output: ['x' 'j' 'e' 'g' 'q' 'q' 'm' 'w' 'a' 'i' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
pred: ['x' 'j' 'e' 'g' 'q' 'q' 'm' 'w' 'a' 'i' '</s>']
-------------
input: ['b' 'c' 'e' 'k' 'y' 'f' 'c' 't' 'm' 'n' 'z' '</s>' '<pad>' '<pad>'
 '<pad>']
output: ['z' 'n' 'm' 't' 'c' 'f' 'y' 'k' 'e' 'c' 'b' '</s>' '<pad>' '<pad>'
 '<pad>']
pred: ['z' 'n' 't' 'c' 'c' 'f' 'y' 'k' 'e' 'c' 'b' '</s>']
-------------
input: ['w' 'z' 'j' 't' 'p' 'a' 'z' 'e' 'x' 'k' 'l' 'k' '</s>' '<pad>' '<pad>']
output: ['k' 'l' 'k' 'x' 'e' 'z' 'a' 'p' 't' 'j' 'z' 'w' '</s>' '<pad>' '<pad>']
pred: ['k' 'l' 'k' 'x' 'e' 'z' 'a'

input: ['i' 'q' 'b' 'y' 'u' 'y' 'o' 'i' 'j' 'l' 'v' '</s>' '<pad>' '<pad>'
 '<pad>']
output: ['v' 'l' 'j' 'i' 'o' 'y' 'u' 'y' 'b' 'q' 'i' '</s>' '<pad>' '<pad>'
 '<pad>']
pred: ['v' 'l' 'j' 'i' 'o' 'y' 'u' 'y' 'b' 'q' 'i' '</s>']
-------------
input: ['j' 'f' 'q' 'z' 'k' 'r' 'x' 'x' 'n' 'i' 'x' '</s>' '<pad>' '<pad>'
 '<pad>']
output: ['x' 'i' 'n' 'x' 'x' 'r' 'k' 'z' 'q' 'f' 'j' '</s>' '<pad>' '<pad>'
 '<pad>']
pred: ['x' 'i' 'n' 'x' 'x' 'r' 'k' 'z' 'q' 'f' 'j' '</s>']
-------------
input: ['m' 'b' 'o' 'z' 'n' 'l' 't' 't' 'n' 'y' 'd' '</s>' '<pad>' '<pad>'
 '<pad>']
output: ['d' 'y' 'n' 't' 't' 'l' 'n' 'z' 'o' 'b' 'm' '</s>' '<pad>' '<pad>'
 '<pad>']
pred: ['d' 'y' 'n' 't' 't' 'l' 'n' 'z' 'o' 'b' 'm' '</s>']
-------------
input: ['b' 'r' 'b' 's' 'i' 'i' 'y' 'y' 'x' 'c' 'n' '</s>' '<pad>' '<pad>'
 '<pad>']
output: ['n' 'c' 'x' 'y' 'y' 'i' 'i' 's' 'b' 'r' 'b' '</s>' '<pad>' '<pad>'
 '<pad>']
pred: ['n' 'c' 'x' 'y' 'y' 'i' 'i' 's' 'b' 'r' 'b' '</s>']
-------------
input: ['u' 'a' 'w' 'l' 

input: ['t' 'y' 's' 'b' 'q' 'n' 's' 'm' 'a' 'u' 'i' 'h' '</s>' '<pad>' '<pad>']
output: ['h' 'i' 'u' 'a' 'm' 's' 'n' 'q' 'b' 's' 'y' 't' '</s>' '<pad>' '<pad>']
pred: ['h' 'i' 'u' 'h' 'm' 's' 'n' 'q' 'b' 's' 'y' 't' '</s>']
-------------
input: ['t' 'g' 'm' 'q' 'h' 'y' 'l' 'j' 'c' 'a' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
output: ['a' 'c' 'j' 'l' 'y' 'h' 'q' 'm' 'g' 't' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
pred: ['a' 'c' 'j' 'l' 'y' 'h' 'q' 'm' 'g' 't' '</s>']
-------------
input: ['n' 'a' 'a' 'o' 'a' 'y' 'k' 'v' 'q' 'j' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
output: ['j' 'q' 'v' 'k' 'y' 'a' 'o' 'a' 'a' 'n' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
pred: ['j' 'q' 'v' 'k' 'y' 'a' 'o' 'a' 'a' 'n' '</s>']
-------------
input: ['a' 'd' 'f' 'v' 'd' 'r' 'x' 'i' 'z' 'y' 'l' 'i' '</s>' '<pad>' '<pad>']
output: ['i' 'l' 'y' 'z' 'i' 'x' 'r' 'd' 'v' 'f' 'd' 'a' '</s>' '<pad>' '<pad>']
pred: ['i' 'l' 'i' 'y' 'i' 'x' 'r' 'd' 'v' 'f' 'd' 'a' '</s>']
-------------
input: ['o' 'k' 'w' 'a' 'l' 

pred: ['w' 'i' 'a' 'n' 'u' 's' 'k' 'd' 'y' 'n' '</s>']
-------------
input: ['y' 'v' 'c' 'g' 'd' 'x' 'v' 'w' 'y' 'w' 'o' 'm' '</s>' '<pad>' '<pad>']
output: ['m' 'o' 'w' 'y' 'w' 'v' 'x' 'd' 'g' 'c' 'v' 'y' '</s>' '<pad>' '<pad>']
pred: ['m' 'o' 'w' 'y' 'w' 'v' 'x' 'd' 'g' 'c' 'v' 'y' '</s>']
-------------
input: ['a' 'n' 'c' 'h' 'y' 'k' 'r' 's' 'e' 'i' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
output: ['i' 'e' 's' 'r' 'k' 'y' 'h' 'c' 'n' 'a' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
pred: ['i' 'e' 's' 'r' 'k' 'y' 'h' 'c' 'n' 'a' '</s>']
-------------
input: ['e' 'c' 'm' 'i' 'e' 'k' 'v' 'j' 'k' 'r' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
output: ['r' 'k' 'j' 'v' 'k' 'e' 'i' 'm' 'c' 'e' '</s>' '<pad>' '<pad>' '<pad>'
 '<pad>']
pred: ['r' 'k' 'j' 'v' 'k' 'e' 'i' 'm' 'c' 'e' '</s>']
-------------
input: ['k' 'b' 'k' 'h' 'm' 't' 'a' 'r' 'y' 's' 'l' '</s>' '<pad>' '<pad>'
 '<pad>']
output: ['l' 's' 'y' 'r' 'a' 't' 'm' 'h' 'k' 'b' 'k' '</s>' '<pad>' '<pad>'
 '<pad>']
pred: ['l' 's' 'y' 'r' 'a'

In [44]:
# plt.matshow(attn_weight.numpy())

In [45]:
attn_weight.shape

torch.Size([13, 15])

In [46]:
input_tensor = make_alphabet_tensor()
output_words, attentions = evaluate(encoder1, decoder1, input_tensor)

In [47]:
def showAttention(input_sentence, output_words, attentions):
    # colorbar로 그림 설정
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # 축 설정
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['</s>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # 매 틱마다 라벨 보여주기
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()

In [48]:
def evaluateAndShowAttention():
    input_tensor = make_alphabet_tensor()
    output_words, attentions = evaluate(
        encoder1, decoder1, input_tensor)
    
    input_sentence = input_tensor[0].tolist()
    input_sentence = list(map(lambda x: i2a[x], input_sentence))
    print('original input =', input_sentence)
    input_length = input_sentence.index('<pad>')
    input_sentence = ' '.join(input_sentence[:input_length])
    print('input  =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions)

In [49]:
evaluateAndShowAttention()
#evaluateAndShowAttention("elle est trop petit .")
#evaluateAndShowAttention("je ne crains pas de mourir .")
#evaluateAndShowAttention("c est un jeune directeur plein de talent .")

original input = ['u', 'r', 's', 'l', 'h', 'q', 'b', 'r', 'n', 'b', 'w', '<pad>', '<pad>', '<pad>', '<pad>']
input  = u r s l h q b r n b w
output = w b r b q h l s r u </s>


/heavy_data/jkfirst/workspace/git/publish/env_pub/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: FixedFormatter should only be used together with FixedLocator
  # Remove the CWD from sys.path while we load stuff.
/heavy_data/jkfirst/workspace/git/publish/env_pub/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: FixedFormatter should only be used together with FixedLocator
  # This is added back by InteractiveShellApp.init_path()


In [50]:
generate_random_alphabet_index()

([8, 12, 8, 8, 16, 9, 13, 27, 15, 18, 10], 11)